Objective of this notebook is to break down the multivariate interpolation layer from the paper, so that I can run physiological time series through it

In [1]:
import argparse
import numpy as np
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as auprc
from sklearn.metrics import roc_auc_score as auc_score
import keras

#from keras.utils import multi_gpu_model
from keras.layers import Input, Dense, GRU, Lambda, Permute, Concatenate
from keras.models import Model
from interpolation_layer import single_channel_interp, cross_channel_interp
from mimic_preprocessing import load_data, trim_los, fix_input_format
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

np.random.seed(10)

## Import helper functions

In [2]:
import sys
sys.path
if '/home/ugrads/n/nickcheng0921/TAMU-MedResearch/' not in sys.path:
    sys.path.append('/home/ugrads/n/nickcheng0921/TAMU-MedResearch/')

In [3]:
from helper import hold_out, mean_imputation

## Arguments

In [15]:
gpu_num = 1
epoch = 1000
hid = 256
ref_points = 128
hours_look_ahead = 48
if gpu_num > 0:
    batch = 512*gpu_num
else:
    batch = 512
    
#nicks notes args
vocabulary = 6000

## Loading Data
adjust # of patients in mimic_preprocessing.py

In [5]:
# Loading dataset
# y : (N,) discrete for classification, real values for regression
# x : (N, D, tn) input multivariate time series data with dimension
#     where N is number of data cases, D is the dimension of
#     sparse and irregularly sampled time series and tn is the union
#     of observed time stamps in all the dimension for a data case n.
#     Since each tn is of variable length, we pad them with zeros to# Loading dataset
# y : (N,) discrete for classification, real values for regression
# x : (N, D, tn) input multivariate time series data with dimension
#     where N is number of data cases, D is the dimension of
#     sparse and irregularly sampled time series and tn is the union
#     of observed time stamps in all the dimension for a data case n.
#     Since each tn is of variable length, we pad them with zeros to
#     have an array representation.
# m : (N, D, tn) where m[i,j,k] = 0 means that x[i,j,k] is not observed.
# T : (N, D, tn) represents the actual time stamps of observation;

vitals, label = load_data(look_ahead_time = hours_look_ahead)
vitals, timestamps = trim_los(vitals, hours_look_ahead)
x, m, T = fix_input_format(vitals, timestamps)
mean_imputation(x, m)
x = np.concatenate((x, m, T, hold_out(m)), axis=1)  # input format
y = np.array(label)
print(f"X shape: {x.shape}, Y shape: {y.shape}")
timestamp = x.shape[2]
num_features = x.shape[1] // 4
#     have an array representation.
# m : (N, D, tn) where m[i,j,k] = 0 means that x[i,j,k] is not observed.
# T : (N, D, tn) represents the actual time stamps of observation;

Loading files ...
Loading Done with 5000 patients! Nick
4532 4532
(4532, 12, 200) 4532
X shape: (4532, 48, 200), Y shape: (4532,)


In [6]:
import pickle
patient_notes = pickle.load(open('notes_5000_'+str(hours_look_ahead)+'hrs.p', 'rb'))
print(len(patient_notes))

4532


# Prediction Model

In [13]:
def customloss(ytrue, ypred):
    """ Autoencoder loss
    """
    # standard deviation of each feature mentioned in paper for MIMIC_III data
    wc = np.array([3.33, 23.27, 5.69, 22.45, 14.75, 2.32,
                   3.75, 1.0, 98.1, 23.41, 59.32, 1.41])
    wc.shape = (1, num_features)
    y = ytrue[:, :num_features, :]
    m2 = ytrue[:, 3*num_features:4*num_features, :]
    m2 = 1 - m2
    m1 = ytrue[:, num_features:2*num_features, :]
    m = m1*m2
    ypred = ypred[:, :num_features, :]
    x = (y - ypred)*(y - ypred)
    x = x*m
    count = tf.reduce_sum(m, axis=2)
    count = tf.where(count > 0, count, tf.ones_like(count))
    x = tf.reduce_sum(x, axis=2)/count
    x = x/(wc**2)  # dividing by standard deviation
    x = tf.reduce_sum(x, axis=1)/num_features
    return tf.reduce_mean(x)

seed = 0
results = {}
results['loss'] = []
results['auc'] = []
results['acc'] = []
results['auprc'] = []

# interpolation-prediction network


def interp_net():
    if gpu_num > 1:
        dev = "/cpu:0"
    else:
        dev = "/gpu:0"
    with tf.device(dev):
        main_input = Input(shape=(4*num_features, timestamp), name='input')
        notes_input = Input(shape=(vocabulary), name='notes_input')
        notes_output = Dense(hid, activation='sigmoid', name='notes_output')(notes_input)
        sci = single_channel_interp(ref_points, hours_look_ahead)
        cci = cross_channel_interp()
        interp = cci(sci(main_input))
        reconst = cci(sci(main_input, reconstruction=True),
                      reconstruction=True)
        aux_output = Lambda(lambda x: x, name='aux_output')(reconst)
        z = Permute((2, 1))(interp)
        z = GRU(hid, activation='tanh', recurrent_dropout=0.2, dropout=0.2)(z)
        merged_input = Concatenate()([notes_output, z])
        #print(f"Z SHAPE {z.shape} NOTES SHAPE {notes_input.shape} MERGED SHAPE {merged_input.shape}")
        main_output = Dense(1, activation='sigmoid', name='main_output')(merged_input)
        orig_model = Model([main_input, notes_input], [main_output, aux_output])
    if gpu_num > 1:
        model = multi_gpu_model(orig_model, gpus=gpu_num)
    else:
        model = orig_model
    #print(orig_model.summary())
    return model

earlystop = keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.0000, patience=20, verbose=0)
callbacks_list = [earlystop]

## Notes vectorizer
Takes input of size n, and vocab of length l.
returns vector of (n, l) where l is vocab embeddings

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "00"

In [14]:
i = 0
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
notes_vectorizer = TfidfVectorizer(max_features=vocabulary)

for train, test in kfold.split(np.zeros(len(y)), y):
    print("Running Fold:", i+1)
    model = interp_net()  # re-initializing every time
    kfold_notes_train = [patient_notes[i] for i in train]
    kfold_notes_test = [patient_notes[i] for i in test]
    notes_tfidf = notes_vectorizer.fit(kfold_notes_train) #train vocab on train set, then use vectorizer on test set
    model.compile(
        optimizer='adam',
        loss={'main_output': 'binary_crossentropy', 'aux_output': customloss},
        loss_weights={'main_output': 1., 'aux_output': 1.},
        metrics={'main_output': 'accuracy'})
    model.fit(
        {'input': x[train], 'notes_input': notes_vectorizer.transform(kfold_notes_train).todense()}, {'main_output': y[train], 'aux_output': x[train]},
        batch_size=batch,
        callbacks=callbacks_list,
        epochs=epoch,
        validation_split=0.20,
        verbose=2)
    y_pred = model.predict({'input': x[test], 'notes_input': notes_vectorizer.transform(kfold_notes_test).todense()}, batch_size=batch)
    y_pred = y_pred[0]
    total_loss, score, reconst_loss, acc = model.evaluate(
        {'input': x[test], 'notes_input': notes_vectorizer.transform(kfold_notes_test).todense()}, {'main_output': y[test], 'aux_output': x[test]},
        batch_size=batch,
        verbose=0)
    results['loss'].append(score)
    results['acc'].append(acc)
    results['auc'].append(auc_score(y[test], y_pred))
    results['auprc'].append(auprc(y[test], y_pred))
    print(results, "RECONST LOSS:", reconst_loss)
    i += 1

Running Fold: 1
6/6 - 7s - loss: 0.7602 - main_output_loss: 0.4262 - aux_output_loss: 0.3341 - main_output_accuracy: 0.8159 - val_loss: 0.5690 - val_main_output_loss: 0.2699 - val_aux_output_loss: 0.2990 - val_main_output_accuracy: 0.9214 - 7s/epoch - 1s/step
2/2 [==============================] - 1s 210ms/step
{'loss': [0.2939535975456238], 'auc': [0.5745614035087719], 'acc': [0.9162072539329529], 'auprc': [0.1738176088858518]} RECONST LOSS: 0.30518466234207153
Running Fold: 2


KeyboardInterrupt: 

In [ ]:
hours_look_ahead

In [ ]:
#output of network on 5000 patients
lah_arr = [6, 12, 18, 24, 30, 36, 42, 48]
acc = [0.9074, 0.9125, 0.9061, 0.9135, 0.9045, 0.9153, 0.9123, .9126]
auc = [0.7194, 0.8047, 0.8186, 0.8336, 0.8433, 0.8322, 0.8290, .8426]
loss =[0.3132, 0.2617, 0.2673, 0.2380, 0.2434, 0.2531, 0.2502, .22944]
auprc = [0, 0, 0, 0, 0, .3488, 0.3076, .3551]

paper_auc = [.8027, .8161, .8138, .8256, .8324, .8320, .8376, .8453]

In [ ]:
import matplotlib.pyplot as plt
plt.title('Output of fusion model on varying admission times for ~5000 patients')
plt.plot(lah_arr, acc, label='acc')
plt.plot(lah_arr, auc, label='auc')
plt.plot(lah_arr, loss, label='loss')
plt.legend(loc='lower right')
plt.xlabel('Notes & time series data used in hrs')
plt.savefig('baseline.png')

In [ ]:
plt.title('Comparison of my model vs baseline from paper')
plt.plot(lah_arr, auc, label='mine')
plt.plot(lah_arr, paper_auc, label='paper')
plt.xlabel('Notes & time series data used in hrs')
plt.legend(loc='lower right')
plt.ylim([0, 1])

In [ ]:
(np.mean(results['acc']), np.mean(results['auc']), np.mean(results['loss']), np.mean(results['auprc']))

In [ ]:
plt.plot(results['acc'], label='acc')
plt.plot(results['auc'], label='auc')
plt.plot(results['loss'], label='loss')
plt.legend(loc="lower right")
plt.title(f"Outputs of interpolation network on 5000 patients")
plt.xlabel("Run from 1-5")